## MLP实现MNIST手写数字识别
MNIST数据集请点击[这里查看](http://yann.lecun.com/exdb/mnist/)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

### 建立数据

In [2]:
mnist = input_data.read_data_sets("data\\MNIST_data\\", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data\MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data\MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data\MNIST_data\t10k-images-idx3-ubyte.gz
Extracting data\MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


* Dropout论文可以[点击这里](http://jmlr.org/papers/v15/srivastava14a.html)  

### 创建MLP基本参数
* 设定输入节点数、隐含层节点数、训练迭代次数这三个超参
* 创建两个tf.Variable 作为W
* 创建两个tf.placeholder 作为B
* 创建三个tf.placeholder 作为数据x，标签label，onehot保留系数keep_prob

In [3]:
input_num = 784
h1_num = 300
epoch = 3000

W1 = tf.Variable(tf.truncated_normal([input_num, h1_num],stddev=0.1))
W2 = tf.Variable(tf.zeros([h1_num,10]))
B1 = tf.Variable(tf.zeros([h1_num]))
B2 = tf.Variable(tf.zeros([10]))
x = tf.placeholder(tf.float32, [None, input_num])
keep_prob = tf.placeholder(tf.float32)
label = tf.placeholder(tf.float32, [None,10])

### 编写模型

In [4]:
# 正向传播
hidden1 = tf.nn.relu(tf.matmul(x,W1) + B1)
hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
y = tf.nn.softmax(tf.matmul(hidden1_drop, W2) + B2)

# 反向传播
cross_entropy = tf.reduce_mean(-tf.reduce_sum(label * tf.log(y), reduction_indices=[1]))
train_step = tf.train.AdagradOptimizer(0.01).minimize(cross_entropy)

### 创建session并开始训练

In [5]:
with tf.Session() as sess:
    # 训练开始
    start = time.time()
    tf.global_variables_initializer().run()
    for _ in range(epoch):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_xs, label: batch_ys, keep_prob:0.75})
    stop = time.time()
    # 训练结束
    
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(label,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    predict_acc = sess.run(accuracy, feed_dict={x: mnist.test.images, label: mnist.test.labels, keep_prob:1.0})
    print("测试集准确度为：{:.2f}%".format(predict_acc * 100))
    print("程序耗时：{:.2f}秒".format(stop-start))

测试集准确度为：93.16%
程序耗时：11.79秒
